# Compute OPR, Ranking point, and Win ratings

In [14]:
from __future__ import print_function
import pickle
#import time
import sys
sys.path.append('..')
import swagger_client as v3client
from swagger_client.rest import ApiException
from collections import Counter
from IPython.display import HTML

In [15]:


HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [16]:

year = 2019
events = ['2019waahs', '2019wasno', '2019pncmp']

filename = 'matches_{}.pkl'.format(year)
teamname = 'teams_{}.pkl'.format(year)
matches = []
with open(filename, 'rb') as f:
    matches = pickle.load(f)


teams = []
with open(teamname, 'rb') as f:
    teams = pickle.load(f)
    
teamlookup = dict([(t.key,t.nickname) for t in teams])
    
print('Loaded {} events and {} teams'.format(len(matches['matches']),len(teams)))

Loaded 303 events and 3807 teams


In [17]:
if (len(events)>0):
    print("Using only events {}".format(','.join(events)))
non_empty = [k for k in matches['matches'].keys() if len(matches['matches'][k])>0]
data = [m for k in matches['matches'] for m in matches['matches'][k] if k in events or len(events)==0]
data = [m for m in data if m.winning_alliance!='']
#print(len(data))
red = [x for m in data for x in m.alliances.red.team_keys]
blue = [x for m in data for x in m.alliances.blue.team_keys]

Counter(red+blue)

# qualifying matches
qualifiers = [m for m in data if m.comp_level=='qm']
alliances = [' '.join(m.alliances.red.team_keys) for m in qualifiers]  + [' '.join(m.alliances.blue.team_keys) for m in qualifiers]
scores = [m.alliances.red.score for m in qualifiers] + [m.alliances.blue.score for m in qualifiers]
rp = [m.score_breakdown and m.score_breakdown['red']['rp'] or 0.0 for m in qualifiers] + [m.score_breakdown and m.score_breakdown['blue']['rp'] or 0.0 for m in qualifiers]
wins = [int(m.winning_alliance=='red') for m in qualifiers] + [int(m.winning_alliance=='blue') for m in qualifiers]

# finals (we'll ignore these for now)
finals = [m for m in data if m.comp_level!='qm']
alliances_finals = [' '.join(m.alliances.red.team_keys) for m in finals]  + [' '.join(m.alliances.blue.team_keys) for m in finals]
scores_finals = [m.alliances.red.score for m in finals] + [m.alliances.blue.score for m in finals]
#len(scores),len(scores_finals)  # should be 2x number of qualifiers
print('{} total matches'.format(len(data)))

Using only events 2019waahs,2019wasno,2019pncmp
313 total matches


In [18]:
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer
import numpy as np
#from scipy.sparse.linalg import spsolve

# two count vectorizers. This transforms the alliance lists into vector encodings
vectorizer = CountVectorizer(max_features=15000, min_df=1, max_df=1.0, stop_words=None)  

X = vectorizer.fit_transform(alliances)
y = np.column_stack([scores, rp, wins])

# we'll ignore for now
X_finals = vectorizer.transform(alliances_finals)
y_finals = scores_finals

#X[1,:].todense()
#np.shape(y)


In [19]:
from scipy.sparse.linalg import lsqr

oprating = lsqr(X,y[:,0])[0]
rprating = lsqr(X,y[:,1])[0]
winrating = lsqr(X,y[:,2])[0]
ratings_all = np.column_stack([oprating, rprating, winrating])
rating_lookup=dict(zip(sorted(vectorizer.vocabulary_,key=lambda x: vectorizer.vocabulary_[x]),ratings_all))
#print(rating_lookup['frc492'])

In [20]:
teamAggregates = {}

def addMatch(team, m):    
    if team not in teamAggregates:       
        # initialize an empty record for the team.
        teamAggregates[team]= {
            'name': teamlookup[team] if team in teamlookup else 'unknown',
            'totalMatches':0, 
            'rpTotal': 0,
            'totalPoints': 0,
            'winCount': 0
        }
            

    alliance = 'blue' if team in m.alliances.blue.team_keys else 'red'
    points = m.score_breakdown[alliance]
    summary = teamAggregates[team]
    # update all of the fields.
    summary['totalMatches']+=1    
    
    summary['rpTotal'] += points['rp']
    summary['totalPoints'] += points['totalPoints']
    summary['winCount'] += int(m.winning_alliance==alliance)

# Currently just using Week0 events.
for m in qualifiers:
    if m.score_breakdown is None:
        continue
    # only aggregate statistics for regional matches- skip the districts or we're cheating.
    #if m.event_key=='2020pncmp':
    #    continue
    for t in m.alliances.red.team_keys:
        addMatch(t,m)
    for t in m.alliances.blue.team_keys:
        addMatch(t,m)
        
# normalize the aggregates
skipped = 0
for t in teamAggregates:
    for k in teamAggregates[t]:
        if k=='totalMatches' or k=='name':
            continue
        teamAggregates[t][k]/=teamAggregates[t]['totalMatches']
    ratings = ['opr', 'rp_rating', 'win_rating']
    if t in rating_lookup:
        for i,r in enumerate(ratings):
            teamAggregates[t][r]=rating_lookup[t][i]        
    else:
        skipped+=1

print('Skipped {} teams with missing data'.format(skipped))

with open('teamStats_tmp_2019.pkl','wb') as f:
    pickle.dump(teamAggregates,f)

Skipped 0 teams with missing data


In [21]:
# Save to tsv for processing in excel.
with open('stats_{}.tsv'.format(year),'w',encoding='utf-8') as outStats:
    outStats.write('{}\n'.format('\t'.join(['team','name','opr','rp_rating','win_rating','rp','points','wins'])))
    for t in teamAggregates:
        k = teamAggregates[t]
        if 'opr' not in k:
            continue
        outStats.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(t,k['name'], k['opr'],k['rp_rating'],k['win_rating'], k['rpTotal'],k['totalPoints'],k['winCount']))

In [22]:
from pandas import DataFrame

df = DataFrame.from_dict(dict(filter(lambda x: 'opr' in x[1],teamAggregates.items())), orient='index') 
df.index.name = 'team'


In [23]:
"""
Copyright 2019, Marek Cermak
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""

def init_datatable_mode():
    """Initialize DataTable mode for pandas DataFrame represenation."""
    import pandas as pd
    from IPython.core.display import display, Javascript

    # configure path to the datatables library using requireJS
    # that way the library will become globally available
    display(Javascript("""
        require.config({
            paths: {
                DT: '//cdn.datatables.net/1.10.19/js/jquery.dataTables.min',
            }
        });
        $('head').append('<link rel="stylesheet" type="text/css" href="//cdn.datatables.net/1.10.19/css/jquery.dataTables.min.css">');
    """))

    def _repr_datatable_(self):
        """Return DataTable representation of pandas DataFrame."""
        # classes for dataframe table (optional)
        classes = ['table', 'table-striped', 'table-bordered']

        # create table DOM
        script = (
            f'$(element).html(`{self.to_html(index=True, classes=classes)}`);\n'
        )

        # execute jQuery to turn table into DataTable
        script += """
            require(["DT"], function(DT) {
                $(document).ready( () => {
                    // Turn existing table into datatable
                    $(element).find("table.dataframe").DataTable();
                })
            });
        """

        return script

    pd.DataFrame._repr_javascript_ = _repr_datatable_

In [24]:
import numpy as np
import pandas as pd
init_datatable_mode()  # initialize [DataTables]
df

<IPython.core.display.Javascript object>

,name,totalMatches,rpTotal,totalPoints,winCount,opr,rp_rating,win_rating
team,,,,,,,,
frc1258,SeaBot,12,1.916667,57.416667,0.666667,19.475149,1.023305,0.435570
frc1294,Top Gun,11,1.454545,55.090909,0.454545,15.298365,0.479407,0.175820
frc1318,Issaquah Robotics Society,24,2.166667,67.583333,0.666667,28.232943,1.110386,0.336298
frc1425,Error Code Xero,12,1.750000,67.166667,0.500000,21.196041,0.498782,0.126434
frc1540,Flaming Chickens,11,1.727273,76.818182,0.454545,31.460213,0.703782,0.170876
frc1778,Chill Out,12,1.833333,62.083333,0.583333,18.234570,0.589437,0.219422
frc1899,Saints Robotics,24,1.375000,61.458333,0.375000,16.423075,0.181469,0.012471
frc1983,Skunk Works Robotics,23,2.173913,71.565217,0.652174,30.583494,1.157238,0.341799
frc2046,Bear Metal,24,2.833333,75.625000,0.875000,34.471003,1.675922,0.481090
